In [1]:
import strat_fig
from pathlib import Path
import numpy as np
import pandas as pd
from moonpies import moonpies as mp
from moonpies import default_config

In [2]:
# Choose cold trap crater
coldtrap = "Haworth"
data_dir = '' # Manually set model result dir, or else get from cfg
seed = '07045'

# Get cfg
cfg_dict = {}
cfg_dict['seed'] = seed
cfg = default_config.Cfg(**cfg_dict)
if not data_dir:
    data_dir = Path(cfg.outpath)

# Input paths
f_ice = data_dir.joinpath('ice_columns_mpies.csv')
f_ej = data_dir.joinpath('ej_columns_mpies.csv')

# Output paths
figbasename = f'strat_{coldtrap}_{cfg.run_name}_{cfg.run_date}'
figpath = Path(cfg.figspath)
keypath = figpath.joinpath(figbasename + '.png')
plotpath = figpath.joinpath(figbasename + '_key.png')

# Read in crater, ice, ej data
crater_csv = mp.read_crater_list(cfg.crater_csv_in, cfg.crater_cols)
ice_df = pd.read_csv(f_ice)
ej_df = pd.read_csv(f_ej)
time_arr = mp.get_time_array(cfg)

In [3]:
thresh = 1e-6
time = time_arr
ice = ice_df.loc[:, coldtrap].values
ejecta = ej_df.loc[:, coldtrap].values
ejecta_sources = ej_df.loc[:, 'ejecta_source'].values

s = mp.make_strat_col(time, ice, {}, ejecta, ejecta_sources, cfg, thresh)
s

,label,time,depth,ice,ejecta,icepct
0,Ice,4.180000e+09,248.527542,47.601868,0.000000,100.0000
1,Shoemaker,4.160000e+09,243.211029,1.829567,20.946774,8.0328
2,Ice,3.990000e+09,169.667633,52.423275,0.000000,100.0000
3,"Amundsen,Idel'son L",3.970000e+09,168.011414,0.227020,9.576299,2.3157
4,Ice,3.890000e+09,154.141266,4.105841,0.000000,100.0000
5,"de Gerlache,Schrodinger",3.870000e+09,154.102249,0.746827,31.530045,2.3138
6,Ice,3.830000e+09,102.114708,26.975426,0.000000,100.0000
7,"Slater,Scott",3.810000e+09,94.849945,6.178198,9.333152,39.8302
8,Cabeus,3.790000e+09,79.338600,0.156514,65.339729,0.2390
9,"Nobile,Wiechert P",3.780000e+09,13.842355,0.280754,12.301815,2.2313


In [11]:
a = {'a': 123, 'b': 456}
a.items()

TypeError: 'dict_items' object is not subscriptable

In [5]:

label = np.empty(len(time), dtype=object)
label[ice > thresh] = 'Ice'
label[ejecta > thresh] = ejecta_sources[ejecta > thresh]
depth = np.cumsum(ice[::-1] + ejecta[::-1])[::-1]
ice_pct = 100 * ice / (ice + ejecta)

ice_sources = {}

# Make DataFrame
data = np.array([time, ice, ejecta, depth, ice_pct]).T
cols = ['time', 'ice', 'ejecta', 'depth', 'ice_pct']
for k, v in ice_sources.items():
    data.append(v)
    cols.append(k)
strat = pd.DataFrame(data, columns=cols, dtype=cfg.dtype)

# Remove empty rows from strat col
strat.insert(0, 'label', label)
strat = strat.dropna(subset=['label']) 

/home/cjtu/projects/moonpies/.venv/lib/python3.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in true_divide
  """


In [6]:
adj_check = (strat.label != strat.label.shift()).cumsum()
agg = {k: 'sum' for k in ('ice', 'ejecta', *ice_sources.keys())}
agg['label'] = 'last'
agg['time'] = 'last'
agg['depth'] = 'last'
agg['ice_pct'] = 'mean'
strat.groupby(['label', adj_check], as_index=False, sort=False).agg(agg)

,ice,ejecta,label,time,depth,ice_pct
0,39.038647,0.000000,Ice,4.160000e+09,224.297775,100.000000
1,1.895733,20.946774,Shoemaker,4.140000e+09,219.889664,8.299148
2,61.045265,0.000000,Ice,3.900000e+09,155.699844,100.000000
3,0.087784,31.530045,Schrodinger,3.860000e+09,136.001892,0.277642
4,0.459292,0.000000,Ice,3.850000e+09,104.384064,100.000000
5,0.493711,9.576299,Amundsen,3.830000e+09,103.924767,4.902789
6,6.252191,65.339729,Cabeus,3.800000e+09,93.854759,8.733095
7,0.540827,0.000000,Ice,3.760000e+09,22.170015,100.000000
8,0.076543,9.333152,"Wiechert,Scott",3.740000e+09,21.722012,0.813444
9,0.010503,12.301815,Nobile,3.720000e+09,12.312318,0.085301


In [21]:
(ice > thresh)

array([False,  True,  True,  True,  True,  True,  True,  True,  True,
        True, False,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
       False, False, False,  True,  True, False,  True, False, False,
        True, False,  True,  True,  True, False,  True, False,  True,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False,

In [3]:
strat = strat_fig.get_strat(ice_df, ej_df)
strat[strat.depth > 170]

/home/cjtu/projects/moonpies/.venv/lib/python3.7/site-packages/pandas/core/indexing.py:1758: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, v)


,time,depth,ice,ejecta,label
11,4.140000e+09,171.574820,2.441798,0.000000,Ice
11,4.140000e+09,171.574820,2.441798,0.000000,Ice
10,4.150000e+09,174.016618,2.830345,0.000000,Ice
10,4.150000e+09,174.016618,2.830345,0.000000,Shoemaker
9,4.160000e+09,176.846964,5.821515,20.946774,Shoemaker
9,4.160000e+09,176.846964,5.821515,20.946774,Ice
7,4.180000e+09,203.615252,7.994002,0.000000,Ice
7,4.180000e+09,203.615252,7.994002,0.000000,Ice
6,4.190000e+09,211.609254,2.813713,0.000000,Ice
6,4.190000e+09,211.609254,2.813713,0.000000,Ice


In [4]:
    adj_check = (strat.label != strat.label.shift()).cumsum()
    distinct_layers = strat.groupby(['label', adj_check], as_index=False,
                                sort=False).agg({
                                    "depth" : 'last',
                                    "time": 'last',
                                    "label": 'last'})
    yticks_depth = distinct_layers.depth.values
    yticks_depth = np.insert(yticks_depth, 0, 0) #Add zero to start
    yticks_time = distinct_layers.time.values
    yticks_time = np.insert(yticks_time, 0, 0)
    print(yticks_time)

[0.00e+00 3.84e+09 3.85e+09 3.89e+09 3.95e+09 3.96e+09 3.98e+09 4.15e+09
 4.16e+09 4.24e+09]


In [5]:
distinct_layers

,depth,time,label
0,0.000000,3.840000e+09,Scott
1,9.340065,3.850000e+09,Schrodinger
2,41.093166,3.890000e+09,"Sverdrup,Nobile"
3,56.543434,3.950000e+09,Ice
4,57.054439,3.960000e+09,Amundsen
5,67.452926,3.980000e+09,Cabeus
6,174.016618,4.150000e+09,Ice
7,176.846964,4.160000e+09,Shoemaker
8,235.777658,4.240000e+09,Ice


In [80]:
def get_strat(ice_df, ej_df, coldtrap="Haworth", thresh=1e-6):
    """Return strat column df of coldtrap from ice_df, ej_df."""
    # Get columns of strat df from ice_df and ej_df
    strat = ice_df[["time"]].copy()
    strat["ice"] = ice_df[coldtrap]
    strat["ejecta"] = ej_df[coldtrap]
    
    # Zero out all thicknesses below thresh
    strat[strat < thresh] = 0

    # Get indices where ice and ejecta exist
    iice = strat.ice > 0
    iej = strat.ejecta > 0
    iboth = iice & iej


    # Get total depth - cumulative sum of ice and ejecta depths
    strat['depth'] = strat.ice.cumsum() + strat.ejecta.cumsum()
    # strat["depth"] = strat.depth.max() - strat.depth

    # Get ice % where ice and ejecta exist (ice / ejecta)
    strat["ice_pct"] = 0
    strat.loc[strat.ice > 0, 'ice_pct'] = 100
    strat.loc[iboth, 'ice_pct'] = 100 * strat.ice[iboth] / strat.ejecta[iboth]

    # Label rows
    strat['label'] = ''
    strat.loc[iej, 'label'] = ej_df[iej].ejecta_source.fillna('')
    strat.loc[iice & ~iej, 'label'] = 'Ice'
    
    # Drop rows with no ice or ejecta
    strat = strat[iice | iej] 

    # Combine adjacent rows with same label into layers
    adj_check = (strat.label != strat.label.shift()).cumsum()
    strat = strat.groupby([
        'label', adj_check], as_index=False, sort=False).agg({
                                    'label': 'last',
                                    'time': 'last',
                                    'depth': 'last',
                                    'ice_pct': 'mean',
                                    'ice': 'sum',
                                    'ejecta': 'sum'})
    # strat.to_csv("strat_output.csv")
    return strat


def get_strat_layers(strat_df, timestart=4.25e9):
    """Return strat_df as unique layers (range start to end) for plotting."""
    d_top = np.insert(strat_df.iloc[:-1].depth.values, 0, 0)
    d_bot = strat_df.iloc[:].depth.values
    t_top = np.insert(strat_df.iloc[:-1].time.values, 0, timestart)
    t_bot = strat_df.iloc[:].time.values
    strat_top = strat_df.set_index(np.arange(0, 2*len(strat_df), 2))
    strat_top['depth'] = d_top
    strat_top['time'] = t_top
    strat_bot = strat_df.copy().set_index(np.arange(1, 2*len(strat_df)+1, 2))
    strat_bot['depth'] = d_bot
    strat_bot['time'] = t_bot
    strat_layers = pd.concat((strat_top, strat_bot)).sort_index()
    return strat_layers

In [58]:
sdf

,label,time,depth,ice_pct,ice,ejecta
0,Ice,4.180000e+09,48.581449,100.000000,48.581449,0.000000
1,Shoemaker,4.160000e+09,75.349737,27.791939,5.821515,20.946774
2,Ice,4.000000e+09,118.383031,100.000000,43.033294,0.000000
3,Cabeus,3.980000e+09,184.743775,1.562639,1.021024,65.339720
4,Amundsen,3.960000e+09,195.142262,8.585648,0.822187,9.576300
5,Ice,3.910000e+09,198.791897,100.000000,3.649635,0.000000
6,"Sverdrup,Nobile",3.890000e+09,211.103535,0.079842,0.009822,12.301816
7,Schrodinger,3.850000e+09,242.856636,0.707437,0.223055,31.530046
8,Scott,3.840000e+09,252.196701,0.074056,0.006912,9.333153


In [81]:
sdf = get_strat(ice_df, ej_df)
sdfr = strat_ranges(sdf)
sdfr

,label,time,depth,ice_pct,ice,ejecta
0,Ice,4.250000e+09,0.000000,100.000000,48.581449,0.000000
1,Ice,4.180000e+09,48.581449,100.000000,48.581449,0.000000
2,Shoemaker,4.180000e+09,48.581449,27.791939,5.821515,20.946774
3,Shoemaker,4.160000e+09,75.349737,27.791939,5.821515,20.946774
4,Ice,4.160000e+09,75.349737,100.000000,43.033294,0.000000
5,Ice,4.000000e+09,118.383031,100.000000,43.033294,0.000000
6,Cabeus,4.000000e+09,118.383031,1.562639,1.021024,65.339720
7,Cabeus,3.980000e+09,184.743775,1.562639,1.021024,65.339720
8,Amundsen,3.980000e+09,184.743775,8.585648,0.822187,9.576300
9,Amundsen,3.960000e+09,195.142262,8.585648,0.822187,9.576300


In [78]:
d_top = np.insert(sdf.iloc[:-1].depth.values, 0, 0)
d_bot = sdf.iloc[:].depth.values
t_top = np.insert(sdf.iloc[:-1].time.values, 0, sdf.time.max())
t_bot = sdf.iloc[:].time.values
strat_top = sdf.set_index(np.arange(0, 2*len(sdf), 2))
strat_top['depth'] = d_top
strat_top['time'] = t_top
strat_bot = sdf.copy().set_index(np.arange(1, 2*len(sdf)+1, 2))
strat_bot['depth'] = d_bot
strat_bot['time'] = t_bot
strat = pd.concat((strat_top, strat_bot)).sort_index()

In [79]:
strat

,label,time,depth,ice_pct,ice,ejecta
0,Ice,4.180000e+09,0.000000,100.000000,48.581449,0.000000
1,Ice,4.180000e+09,48.581449,100.000000,48.581449,0.000000
2,Shoemaker,4.180000e+09,48.581449,27.791939,5.821515,20.946774
3,Shoemaker,4.160000e+09,75.349737,27.791939,5.821515,20.946774
4,Ice,4.160000e+09,75.349737,100.000000,43.033294,0.000000
5,Ice,4.000000e+09,118.383031,100.000000,43.033294,0.000000
6,Cabeus,4.000000e+09,118.383031,1.562639,1.021024,65.339720
7,Cabeus,3.980000e+09,184.743775,1.562639,1.021024,65.339720
8,Amundsen,3.980000e+09,184.743775,8.585648,0.822187,9.576300
9,Amundsen,3.960000e+09,195.142262,8.585648,0.822187,9.576300


In [49]:
sources = ej_df.ejecta_source.fillna('')
sources[sources != '']

6                   Haworth
9                 Shoemaker
17                 Faustini
25     Hedervari,Idel'son L
27                   Cabeus
29                 Amundsen
31                Unnamed 1
34              de Gerlache
36          Sverdrup,Nobile
40              Schrodinger
41                    Scott
45                 Cabeus B
47               Wiechert P
48                  Scott E
50                   Slater
52                 Wiechert
61                Unnamed 2
96               Wiechert U
102                Idel'son
103              Wiechert J
107              Shackleton
122               Unnamed 3
233              Amundsen C
Name: ejecta_source, dtype: object

In [18]:
sources.index

RangeIndex(start=0, stop=425, step=1)

In [19]:
ice_df.time[sources[sources != ''].index]

6      4.190000e+09
9      4.160000e+09
17     4.080000e+09
25     4.000000e+09
27     3.980000e+09
29     3.960000e+09
31     3.940000e+09
34     3.910000e+09
36     3.890000e+09
40     3.850000e+09
41     3.840000e+09
45     3.800000e+09
47     3.780000e+09
48     3.770000e+09
50     3.750000e+09
52     3.730000e+09
61     3.640000e+09
96     3.290000e+09
102    3.230000e+09
103    3.220000e+09
107    3.180000e+09
122    3.030000e+09
233    1.920000e+09
Name: time, dtype: float64

In [17]:
crater_csv

,cname,lat,lon,diam,age,age_low,age_upp,psr_area,age_ref,prio,notes,rad,x,y,dist2pole
0,Haworth,-87.5,354.8,51400.0,4.180000e+09,20000000.0,20000000.0,1.017932e+09,Cannon et al. 2020,1,NaN,25700.0,-6868.528787,75472.425541,75808.376060
1,Shoemaker,-88.1,45.9,51800.0,4.150000e+09,20000000.0,20000000.0,1.075518e+09,Cannon et al. 2020,1,NaN,25900.0,41366.808609,40087.226377,57614.365806
2,Faustini,-87.2,84.3,42500.0,4.100000e+09,30000000.0,30000000.0,6.639340e+08,Cannon et al. 2020,1,NaN,21250.0,84451.948404,8429.425083,84905.381188
3,Amundsen,-84.4,83.1,103400.0,3.900000e+09,100000000.0,100000000.0,7.019590e+08,Cannon et al. 2020,1,NaN,51700.0,168312.605953,20368.063334,169810.762375
4,Cabeus B,-82.3,305.4,59600.0,3.900000e+09,100000000.0,100000000.0,3.872050e+08,Cannon et al. 2020,2,NaN,29800.0,-189751.642076,134849.472992,233489.798266
5,de Gerlache,-88.5,271.7,32700.0,3.900000e+09,100000000.0,100000000.0,2.432920e+08,Cannon et al. 2020,1,NaN,16350.0,-45459.812503,1349.215737,45485.025636
6,Hedervari,-81.9,85.6,74100.0,3.900000e+09,100000000.0,100000000.0,NaN,Cannon et al. 2020,2,NaN,37050.0,244080.308807,18780.956208,245619.138436
7,Idel'son L,-84.0,118.6,28000.0,3.900000e+09,100000000.0,100000000.0,3.267790e+08,Cannon et al. 2020,2,NaN,14000.0,159448.514533,-86934.152260,181940.102545
8,Unnamed 1,-83.7,69.2,57700.0,3.900000e+09,100000000.0,100000000.0,NaN,Cannon et al. 2020,2,NaN,28850.0,178226.751392,67701.991831,191037.107672
9,Cabeus,-85.3,317.9,100600.0,3.880000e+09,100000000.0,100000000.0,3.150290e+08,Cannon et al. 2020,1,NaN,50300.0,-95441.909939,105627.654673,142519.746994
